John Shamshoian

Homework 2

## Q1

Consider the numerical task of estimating an $n \times n$ kinship matrix $\Phi$ from an $n \times m$ genotype matrix $\mathbf{X}$. Here $n$ is the number of individuals and $m$ is the number of genetic markers. [Lange et al](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4062304/) derived a method of moment estimator of form
$$
    \widehat \Phi_{ij} = \frac{e_{ij} - \sum_{k=1}^m [p_k^2 + (1 - p_k)^2]}{m - \sum_{k=1}^m [p_k^2 + (1 - p_k)^2]}, \quad 1 \le i, j \le n,
$$
where
$$
\begin{eqnarray*}
    e_{ij} &=& \frac{1}{4} \sum_{k=1}^m [x_{ik} x_{jk} + (2 - x_{ik})(2 - x_{jk})] \\
    p_k &=& \frac {1}{2n} \sum_{i=1}^n x_{ik}.
\end{eqnarray*}
$$

0. Write a function that takes a matrix `X` as input and outputs the method of moment estimator
    ```julia
    function kinship(X::Matrix{Float64})
        # your code here
        return Φ
    end
    ```
Make your function as efficient as possible.    

0. In a typical genetic data set, $n$ is at order of $10^3 \sim 10^5$ and $m$ is at order of $10^5 \sim 10^6$. Benchmark your function using a smaller data set
    ```julia
    srand(280) # seed
    X = rand(0.0:2.0, 1000, 10000)
    ```
Efficiency (both speed and memory) will be the most important criterion when grading this question.

In [1]:
srand(280) # seed
X = rand(0.0:2.0, 1000, 10000);

In [2]:
using BenchmarkTools

In [3]:
function kinship(X::Matrix{Float64})
    
    # Get dimensions of the input matrix. 
    n = size(X, 1)
    m = size(X, 2)
    
    # Create matrix to hold phi. Preallocating memory is a good thing
    phi = Matrix{Float64}(n, n)
    p = sum(X, 1) / (2.0 * n)
    
    # Use sumabs2 to quickly compute sum of squared components in a vector
    pnorm = sumabs2(p)
    pnorm2 = sumabs2(1 - p)
    
    # Use this to hold row-sums of X
    taco = sum(X, 2)
    
    # Use BLAS to take advantage of symmetry when multiplying matrices. 
    # Much more efficient than writing X * X'.
    # This is an intermediate step in creating the 'e' matrix. 
    # I don't explicitly create the e matrix because
    # that would cost more memory. Overwriting is better in terms of
    # memory and speed.
    phi = 1.0 / 4.0 * (Base.LinAlg.BLAS.syr!('L', 4.0 * m, ones(n),
                Base.LinAlg.BLAS.syrk('L', 'N', 2.0, X)))
    
    # Overwrite phi to create the lower triangular part of phi
    for j = 1:n
        for i = j:n
            phi[i, j] = (phi[i, j] - 1.0 / 2.0 * (taco[i] + taco[j]) - 
                pnorm - pnorm2) / (m - pnorm - pnorm2)
        end
    end
    
    # Fill in the upper triangular part of phi. Phi is symmetric 
    # so I can loop this way.
    for j = 2:n
        for i = 1:j
            phi[i, j] = phi[j, i]
        end
    end
    phi
end

kinship (generic function with 1 method)

In [5]:
@benchmark kinship(X)

BenchmarkTools.Trial: 
  memory estimate:  23.13 MiB
  allocs estimate:  22
  --------------
  minimum time:     133.339 ms (0.21% GC)
  median time:      148.170 ms (0.95% GC)
  mean time:        148.076 ms (0.91% GC)
  maximum time:     171.237 ms (0.79% GC)
  --------------
  samples:          34
  evals/sample:     1

My kinship function takes about 150ms on my computer and does little garbage collecting. Dr. Zhou's kinship function took 90ms but his CPU is faster.

## Q2

0. Show the **Sherman-Morrison formula**
$$
	(\mathbf{A} + \mathbf{u} \mathbf{u}^T)^{-1} = \mathbf{A}^{-1} - \frac{1}{1 + \mathbf{u}^T \mathbf{A}^{-1} \mathbf{u}} \mathbf{A}^{-1} \mathbf{u} \mathbf{u}^T \mathbf{A}^{-1},
$$
where $\mathbf{A} \in \mathbb{R}^{n \times n}$ is nonsingular and $\mathbf{u} \in \mathbb{R}^n$. This formula supplies the inverse of the symmetric, rank-one  perturbation of $\mathbf{A}$.

0. Show the **Woodbury formula**
$$
	(\mathbf{A} + \mathbf{U} \mathbf{V}^T)^{-1} = \mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1},
$$
where $\mathbf{A} \in \mathbb{R}^{n \times n}$ is nonsingular, $\mathbf{U}, \mathbf{V} \in \mathbb{R}^{n \times m}$, and $\mathbf{I}_m$ is the $m \times m$ identity matrix. In many applications $m$ is much smaller than $n$. Woodbury formula generalizes Sherman-Morrison and is valuable because the smaller matrix $\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U}$ is cheaper to invert than the larger matrix $\mathbf{A} + \mathbf{U} \mathbf{V}^T$.

0. Show the **binomial inversion formula**
$$
	(\mathbf{A} + \mathbf{U} \mathbf{B} \mathbf{V}^T)^{-1} = \mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{B}^{-1} + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1},
$$
where $\mathbf{A}$ and $\mathbf{B}$ are nonsingular.

0. Show the identity
$$
	\text{det}(\mathbf{A} + \mathbf{U} \mathbf{V}^T) = \text{det}(\mathbf{A}) \text{det}(\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U}).
$$
This formula is useful for evaluating the density of a multivariate normal with covariance matrix $\mathbf{A} + \mathbf{U} \mathbf{V}^T$.

1. Assume $(\mathbf{A} + \mathbf{uu}^{T})$ is invertible. I will show the Sherman-Morrison formula satisfies the properties of an inverse.
$$
\begin{align}
(\mathbf{A} + \mathbf{uu}^{T})\bigg(\mathbf{A}^{-1} - \frac{1}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}\bigg) &=\mathbf{I} - \frac{\mathbf{uu}^{T}\mathbf{A}^{-1}}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}} + \mathbf{uu}^{T}A^{-1}\mathbf{} -\frac{\mathbf{uu}^{T}\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}}{1 + \mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}\\
&=\mathbf{I} - \frac{\mathbf{uu}^{T}\mathbf{A}^{-1}}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}} + \frac{\mathbf{uu}^{T}A^{-1}+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}\mathbf{uu}^{T}\mathbf{A}^{-1}}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}} -\frac{\mathbf{uu}^{T}\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}}{1 + \mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}\\
&=\mathbf{I} + \frac{\mathbf{uu}^{T}\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}-\frac{\mathbf{uu}^{T}\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}}{1 + \mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}\\
&=\mathbf{I}
\end{align}
$$
This shows the Sherman-Morrison formula is a right-inverse. I need to check it's a left-inverse as well.
$$
\begin{align}
\bigg(\mathbf{A}^{-1} - \frac{1}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}\bigg) (\mathbf{A} + \mathbf{uu}^{T})&=\mathbf{I} + \mathbf{A}^{-1}\mathbf{uu}^{T} - \frac{\mathbf{A}^{-1}\mathbf{uu}^{T}}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}} - \frac{\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}\mathbf{uu}^{T}}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}\\
&=\mathbf{I} + \frac{\mathbf{A}^{-1}\mathbf{uu}^{T}+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}\mathbf{A}^{-1}\mathbf{uu}^{T}-\mathbf{A}^{-1}\mathbf{uu}^{T}-\mathbf{A}^{-1}\mathbf{uu}^{T}\mathbf{A}^{-1}\mathbf{uu}^{T}}{1+\mathbf{u}^{T}\mathbf{A}^{-1}\mathbf{u}}\\
&= \mathbf{I}
\end{align}
$$

 By definition of inverse,  $(\mathbf{A} + \mathbf{uu}^{T})^{-1}=\mathbf{A}^{-1} - \displaystyle\frac{1}{1 + \mathbf{u}^T \mathbf{A}^{-1} \mathbf{u}} \mathbf{A}^{-1} \mathbf{u} \mathbf{u}^T \mathbf{A}^{-1}$
 
2. In a similar manner,
$$
\begin{align}
(\mathbf{A} + \mathbf{UV}^{T})(\mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1})&=\mathbf{I_n} + \mathbf{UV}^{T}\mathbf{A}^{-1}-\mathbf{U}(\mathbf{I_m}
+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1} - \mathbf{UV}^{T}\mathbf{A}^{-1}\mathbf{U}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})\mathbf{V}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n}+\mathbf{UV}^{T}\mathbf{A}^{-1}-(\mathbf{U}+\mathbf{UV}^{T}\mathbf{A}^{-1}\mathbf{U})(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n} + \mathbf{UV}^{T}\mathbf{A}^{-1}-\mathbf{U}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n}+\mathbf{UV}^{T}\mathbf{A}^{-1}-\mathbf{U}\mathbf{V}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n}
\end{align}
$$
 
 This shows the Woodbury formula is a right inverse for $\mathbf{A} + \mathbf{UV}^{T}$. To show it's a left inverse:
$$
\begin{align}
(\mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1})(\mathbf{A} + \mathbf{UV}^{T})&= \mathbf{I_n} + \mathbf{A}^{-1}\mathbf{UV}^{T}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{UV}^{T}\\
&=\mathbf{I_n}+\mathbf{A}^{-1}\mathbf{UV}^{T}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}(\mathbf{V}^{T}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{UV}^{T})\\
&=\mathbf{I_n}+\mathbf{A}^{-1}\mathbf{UV}^{T}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})\mathbf{V}^{T}\\
&=\mathbf{I_n}+\mathbf{A}^{-1}\mathbf{UV}^{T}-\mathbf{A}^{-1}\mathbf{UV}^{T}\\
&=\mathbf{I_n}
\end{align}
$$

 By definition of inverse, $(\mathbf{A} + \mathbf{UV}^{T})^{-1}=(\mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{I}_m + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1})$
 
3. Continuing, 
$$
\begin{align}
(\mathbf{A}+\mathbf{UBV}^{T})(\mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{B}^{-1} + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1})&= \mathbf{I_n}-\mathbf{U}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}+\mathbf{UBV}^{T}\mathbf{A}^{-1}-\mathbf{UBV}^{T}\mathbf{A}^{-1}\mathbf{U}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n}+\mathbf{UBV}^{T}\mathbf{A}^{-1}-(\mathbf{U}+\mathbf{UBV}^{T}\mathbf{A}^{-1}\mathbf{U})(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n}+\mathbf{UBV}^{T}\mathbf{A}^{-1}-\mathbf{UB}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n}+\mathbf{UBV}^{T}\mathbf{A}^{-1}-\mathbf{UBV}^{T}\mathbf{A}^{-1}\\
&=\mathbf{I_n}
\end{align}
$$

 Left inverse:
$$
\begin{align}
(\mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{B}^{-1} + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1})(\mathbf{A}+\mathbf{UBV}^{T})&=\mathbf{I_n} + \mathbf{A}^{-1}\mathbf{UBV}^{T}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{UBV}^{T}\\
&=\mathbf{I_n}+\mathbf{A}^{-1}\mathbf{UBV}^{T}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}(\mathbf{V}^{T}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{UBV}^{T})\\
&=\mathbf{I_n}+\mathbf{A}^{-1}\mathbf{UBV}^{T}-\mathbf{A}^{-1}\mathbf{U}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})^{-1}(\mathbf{B}^{-1}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})\mathbf{BV}^{T}\\
&=\mathbf{I_n}+\mathbf{A}^{-1}\mathbf{UBV}^{T}-\mathbf{A}^{-1}\mathbf{UBV}^{T}\\
&=\mathbf{I_n}
\end{align}
$$
 This shows $(\mathbf{A}+\mathbf{UBV}^{T})^{-1} = \mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{B}^{-1} + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1}$
 
4. Observate that
 $$
\begin{align}
\begin{pmatrix}\mathbf{I_n} & \mathbf{U}\\\mathbf{0_{m\times n}} & \mathbf{I_m}+\mathbf{V}^{T}\mathbf{U}\end{pmatrix} = \begin{pmatrix}\mathbf{I_n} & \mathbf{0_{n\times m}}\\\mathbf{V}^{T} & \mathbf{I_m}\end{pmatrix}\begin{pmatrix}\mathbf{I_n}+\mathbf{U}\mathbf{V}^{T} & \mathbf{U}\\\mathbf{0_{m\times n}} & \mathbf{I_m}\end{pmatrix}\begin{pmatrix}\mathbf{I_n} & \mathbf{0_{n\times m}}\\-\mathbf{V}^{T} & \mathbf{I_m}\end{pmatrix}
\end{align}
$$

 The determinant of a product is the product of determinants. Thus, $\text{det}(\mathbf{I_n} + \mathbf{UV}^{T}) = \text{det}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{U})$
 
 Therefore
$$
\begin{align}
\text{det}(\mathbf{A}+\mathbf{U}\mathbf{V}^{T}) &=\text{det}(\mathbf{A}+\mathbf{A}\mathbf{A}^{-1}\mathbf{UV}^{T})\\
&=\text{det}(\mathbf{A})\text{det}(\mathbf{I_n}+\mathbf{A}^{-1}\mathbf{UV}^{T})\\
&=\text{det}(\mathbf{A})\text{det}(\mathbf{I_m}+\mathbf{V}^{T}\mathbf{A}^{-1}\mathbf{U})
\end{align}
$$

## Q3

Consider a mixed effects model
$$
	y_i = \mathbf{x}_i^T \beta + \mathbf{z}_i^T \gamma + \epsilon_i, \quad i=1,\ldots,n,
$$
where $\epsilon_i$ are independent normal errors $N(0,\sigma_0^2)$, $\beta \in \mathbb{R}^p$ are fixed effects, and $\gamma \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \sigma_1^2 \mathbf{I}_q$) independent of $\epsilon_i$. 

0. Show that 
$$
    \mathbf{y} \sim N \left( \mathbf{X} \beta, \sigma_0^2 \mathbf{I}_n + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T \right),
$$
where $\mathbf{y} \in \mathbb{R}^n$, $\mathbf{X} \in \mathbb{R}^{n \times p}$, and $\mathbf{Z} \in \mathbb{R}^{n \times q}$. 

0. Write a function, with interface 
    ```julia
    logpdf_mvn(y, Z, σ0, σ1),
    ```
that evaluates the log-density of a multivariate normal with mean $\mathbf{0}$ and covariance $\sigma_0^2 \mathbf{I} + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T$ at $\mathbf{y}$. Make your code efficient in the $n \gg q$ case. 

0. Compare your result (both accuracy and timing) to the [Distributions.jl](http://distributionsjl.readthedocs.io/en/latest/multivariate.html#multivariate-normal-distribution) package using following data.  
    ```julia
    using BenchmarkTools, Distributions

    srand(280)
    n, q = 2000, 10
    Z = randn(n, q)
    σ0, σ1 = 0.5, 2.0
    Σ = σ1^2 * Z * Z.' + σ0^2 * I
    mvn = MvNormal(Σ) # MVN(0, Σ)
    y = rand(mvn) # generate one instance from MNV(0, Σ)

    # check you answer matches that from Distributions.jl
    @show logpdf_mvn(y, Z, σ0, σ1)
    @show logpdf(mvn, y)

    # benchmark
    @benchmark logpdf_mvn(y, Z, σ0, σ1)
    @benchmark logpdf(mvn, y)
    ```

1. From the problem formulation, $\mathbf{Z}\gamma \sim N_n(0, \mathbf{Z}\sigma^2_1\mathbf{Z}^{T})$ and $\varepsilon \sim N_n(0,\sigma^2_0\mathbf{I_n})$ with $\mathbf{Z}\gamma$ independent of $\varepsilon$

 Let $\tilde{\varepsilon} = \mathbf{Z}\gamma + \varepsilon$. This is the sum of two independent multivariate normal variables, so $\tilde{\varepsilon} \sim N_n(0, \sigma^2_0\mathbf{I_n} + \sigma^2_1\mathbf{Z}\mathbf{Z}^{T})$
 
 Therefore $\mathbf{y} = \mathbf{X}\beta + \tilde{\varepsilon} \sim N(\mathbf{X}\beta, \sigma^2_0\mathbf{I_n} + \sigma^2_1\mathbf{Z}\mathbf{Z}^{T})$

In [6]:
# Question 3, part 2

using BenchmarkTools, Distributions

srand(280)
n, q = 2000, 10
Z = randn(n, q)
σ0, σ1 = 0.5, 2.0
Σ = σ1^2 * Z * Z.' + σ0^2 * I
mvn = MvNormal(Σ) # MVN(0, Σ)
y = rand(mvn); # generate one instance from MNV(0, Σ)

In [14]:
function logpdf_mvn(y::Array{Float64,1}, Z::Array{Float64,2},
        σ0::Float64, σ1::Float64)
    
    k = length(y)
    
    # Using matrix determinant lemma
    decomp1 = (4000 * log(σ0) + logdet(I + σ1^2 / σ0^2 * Z' * Z))
    
    # Using Woodbury formula
    Σchol = cholfact(I + σ1^2 / σ0^2 * Z' * Z)
    decomp2 = (1/σ0^2 * sumabs2(y) - σ1^2 / σ0^4 * sumabs2(Σchol[:L] \ Z' * y))
    
    # Multivariate normal pdf
    - (k//2) * log(2π) - (1//2) * decomp1 - (1//2) * decomp2
end

logpdf_mvn (generic function with 1 method)

Float64, 1}, Array{Float64, 2}, Float64, Float64) in module Main at In[11]:2 overwritten at In[14]:4.


In [15]:
@show logpdf_mvn(y, Z, σ0, σ1)
@show logpdf(mvn, y)

logpdf_mvn(y,Z,σ0,σ1) = -1571.5736734653365
logpdf(mvn,y) = -1571.5736734654183


-1571.5736734654183

In [16]:
@benchmark logpdf_mvn(y, Z, σ0, σ1)

BenchmarkTools.Trial: 
  memory estimate:  944.81 KiB
  allocs estimate:  36
  --------------
  minimum time:     253.156 μs (0.00% GC)
  median time:      371.959 μs (0.00% GC)
  mean time:        454.665 μs (15.51% GC)
  maximum time:     4.218 ms (45.44% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [17]:
@benchmark logpdf(mvn, y)

BenchmarkTools.Trial: 
  memory estimate:  15.78 KiB
  allocs estimate:  3
  --------------
  minimum time:     5.839 ms (0.00% GC)
  median time:      6.001 ms (0.00% GC)
  mean time:        6.042 ms (0.00% GC)
  maximum time:     7.621 ms (0.00% GC)
  --------------
  samples:          822
  evals/sample:     1

# Q3.3

I exploited the special structure of the covariance matrix by using the matrix determinant lemma and Woodbury formula. The timing of my function is 14 times faster on average than the <code>logpdf</code> function. However, my function requires more overhead because I'm not passing a simple multivariate normal density to be evaluated as in the Distributions package. The evaluated densites are equal up to the ninth decimal point. 